In [2]:
import librosa
import os
from glob import glob
import pandas as pd
import numpy as np
import pickle
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# 1 Aquisição dos dados

In [3]:
path_test = 'dataset/test'
path_train = 'dataset/train'

In [4]:
dataset_test = sorted(glob(path_test + "/*/*", recursive = True))
print(len(dataset_test))
dataset_test[:10]

67


['dataset/test/cat/cat_110.wav',
 'dataset/test/cat/cat_112.wav',
 'dataset/test/cat/cat_115.wav',
 'dataset/test/cat/cat_126.wav',
 'dataset/test/cat/cat_129.wav',
 'dataset/test/cat/cat_130.wav',
 'dataset/test/cat/cat_133.wav',
 'dataset/test/cat/cat_135.wav',
 'dataset/test/cat/cat_137.wav',
 'dataset/test/cat/cat_14.wav']

In [5]:
dataset_train = sorted(glob(path_train + "/*/*", recursive = True))
print(len(dataset_train))
dataset_train[:10]

210


['dataset/train/cat/cat_1.wav',
 'dataset/train/cat/cat_10.wav',
 'dataset/train/cat/cat_100.wav',
 'dataset/train/cat/cat_101.wav',
 'dataset/train/cat/cat_102.wav',
 'dataset/train/cat/cat_103.wav',
 'dataset/train/cat/cat_105.wav',
 'dataset/train/cat/cat_106.wav',
 'dataset/train/cat/cat_107.wav',
 'dataset/train/cat/cat_108.wav']

In [6]:
dataset = dataset_test+dataset_train
len(dataset)

277

In [7]:
#gato
audio, sr = librosa.load(dataset[0])
Audio(data=audio, rate=sr)

In [8]:
#cachorro
audio, sr = librosa.load(dataset[276])
Audio(data=audio, rate=sr)

In [152]:
#cat = 1, dog = 0
labels = []
for d in dataset:
    if 'cat' in d:
        label = 1
    else:
        label = 0
    
    labels.append(label)

print(len(labels))
labels[:20]

277


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# 2 Extração MFCCs

In [34]:
def extract_features_mfcc(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc = 40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("MFCC => Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccsscaled

In [39]:
mfccs = []
for d in dataset:
    mfccs.append(extract_features_mfcc(d))
    
mfccs[:5]

[array([-223.58583   ,   97.86055   ,  -48.041256  ,   78.64677   ,
         -15.693192  ,   29.422092  ,   -9.034269  ,   23.103155  ,
           3.392147  ,  -12.55179   ,   14.84864   ,   -0.9948145 ,
          29.982683  ,   11.474641  ,   23.975592  ,   13.14733   ,
         -19.51557   ,  -10.082049  ,   18.514353  ,    6.465347  ,
         -18.1994    ,    2.357057  ,   -2.8104014 ,    2.5329816 ,
          10.518736  ,    9.630194  ,    3.8638232 ,    4.552205  ,
           4.5010915 ,  -11.024111  ,   -1.3928064 ,    0.62647635,
          -4.4275975 ,   -2.4080257 ,    0.70536137,   -1.5866181 ,
           2.0181923 ,    7.6570997 ,   -0.9881244 ,    1.8574971 ],
       dtype=float32),
 array([-2.1957870e+02,  1.8449553e+02, -6.1977974e+01, -1.3098738e+01,
         3.2448017e+01, -3.9626436e+00, -1.9212191e+01,  8.2762585e+00,
         6.3196158e+00, -1.4249813e+01, -2.7347827e+00,  2.0240194e-01,
        -2.2266073e+00, -5.6498222e+00, -6.1622229e+00,  2.9915035e-01,
        

In [40]:
len(mfccs)

277

In [49]:
mfccs = np.array(mfccs)
labels = np.array(labels)

In [50]:
pickle.dump(mfccs, open('handcrafted_features.pickle', 'wb'))
pickle.dump(labels, open('handcrafted_labels.pickle', 'wb'))   

# 3 Divisão treino/teste

In [53]:
x_train, x_test, y_train, y_test = train_test_split(mfccs,labels, test_size=0.20, random_state=42)

In [54]:
#embaralhando os dados
x_train, y_train = shuffle(x_train, y_train)
x_test, y_test = shuffle(x_test, y_test)

In [55]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(221, 40)
(221,)
(56, 40)
(56,)


# 4 Classificação Random Forest

## 4.1 Parametros default

In [153]:
clf = RandomForestClassifier()#grid seach

In [154]:
clf.fit(x_train, y_train)

RandomForestClassifier()

In [155]:
pred = clf.predict(x_test)
print('Acc = ',accuracy_score(y_test, pred))
print('Kappa = ',cohen_kappa_score(y_test, pred))

Acc =  0.8035714285714286
Kappa =  0.6010362694300518


## 4.2 Parametros gridserch

In [62]:
#random forest
classifier = RandomForestClassifier()
param_grid = {'n_estimators': [90,100,110,120]}
search = GridSearchCV(classifier,
                      param_grid,
                      scoring = 'accuracy',
                      cv = 10,
                      n_jobs = -1)
search.fit(x_train, y_train)
print(search.best_params_)

{'n_estimators': 120}


In [112]:
clf = RandomForestClassifier(n_estimators = 120)#grid seach

In [117]:
clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=120)

In [89]:
pred = clf.predict(x_test)
print('Acc = ',accuracy_score(y_test, pred))
print('Kappa = ',cohen_kappa_score(y_test, pred))

Acc =  0.8392857142857143
Kappa =  0.6735751295336787


# 5 Classificação SVM

## 5.1 Parametros default

In [156]:
clf = SVC()

In [157]:
clf.fit(x_train, y_train)

SVC()

In [158]:
pred = clf.predict(x_test)
print('Acc = ',accuracy_score(y_test, pred))
print('Kappa = ',cohen_kappa_score(y_test, pred))

Acc =  0.8571428571428571
Kappa =  0.7040951122853368


## 5.2 Parametros gridserch

In [159]:
#svm
classifier = SVC()
param_grid = {'kernel':('linear', 'poly', 'sigmoid'),'C':[0.1,0.5,0.7,1.0,1.2]}
search = GridSearchCV(classifier,
                      param_grid,
                      scoring = 'accuracy',cv=5,
                      n_jobs = -1)
search.fit(x_train,y_train)
print(search.best_params_)

{'C': 0.7, 'kernel': 'poly'}


In [162]:
clf = SVC(C=0.7, kernel='poly')

In [163]:
clf.fit(x_train, y_train)

SVC(C=0.7, kernel='poly')

In [164]:
pred = clf.predict(x_test)
print('Acc = ',accuracy_score(y_test, pred))
print('Kappa = ',cohen_kappa_score(y_test, pred))

Acc =  0.8571428571428571
Kappa =  0.7040951122853368
